# Introdução ao Django: Python 

[Aprenda Python com Jupyter](https://github.com/jeanto/python_django_course_notebook) by [Jean Nunes](https://jeanto.github.io/jeannunes)   
Code license: [GNU-GPL v3](https://www.gnu.org/licenses/gpl-3.0.en.html)

---

![Arquitetura MTV do Django](mvt.png)

# **1. Principais Arquivos e Diretórios de um Projeto Django**

Ao criar um projeto Django, a estrutura de diretórios e arquivos é organizada para facilitar o desenvolvimento e a manutenção. Veja os principais componentes:

## Estrutura Básica

```
meuprojeto/
│
├── manage.py
├── meuprojeto/
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   ├── asgi.py
│   └── wsgi.py
└── meuapp/
    ├── __init__.py
    ├── admin.py
    ├── apps.py
    ├── migrations/
    │   └── __init__.py
    ├── models.py
    ├── tests.py
    ├── views.py
    └── templates/
```

### Descrição dos principais arquivos e pastas:

- **manage.py**: Script utilitário para comandos administrativos (rodar servidor, migrações, criar superusuário, etc).
- **meuprojeto/**: Diretório do projeto, contém configurações globais.
  - **settings.py**: Configurações do projeto (banco de dados, apps instalados, etc).
  - **urls.py**: Arquivo principal de rotas do projeto.
  - **asgi.py / wsgi.py**: Pontos de entrada para servidores ASGI/WSGI.
- **meuapp/**: Diretório de um app Django (você pode ter vários apps).
  - **models.py**: Definição dos modelos (tabelas do banco).
  - **views.py**: Definição das views (lógica de resposta às requisições).
  - **admin.py**: Registro dos modelos no admin do Django.
  - **apps.py**: Configurações do app.
  - **migrations/**: Arquivos de migração do banco de dados.
  - **templates/**: Templates HTML do app.




### Conceito de Rotas (URLs) no Django

No Django, as **rotas** (ou URLs) determinam qual view será chamada para cada endereço acessado pelo usuário. O roteamento é feito nos arquivos `urls.py`.

#### Exemplo de urls.py do projeto

```python
# meuprojeto/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('meuapp.urls')),  # Inclui as rotas do app
]
```

#### Exemplo de urls.py do app

```python
# meuapp/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
    path('doadores/', views.listar_doadores, name='listar_doadores'),
    path('doadores/<int:id>/', views.detalhe_doador, name='detalhe_doador'),
]
```

- `''` → Chama a view `index` para a página inicial.
- `'doadores/'` → Chama a view `listar_doadores` para listar todos os doadores.
- `'doadores/<int:id>/'` → Chama a view `detalhe_doador` para mostrar detalhes de um doador específico.

#### Como funciona o fluxo de rotas

1. O usuário acessa uma URL.
2. O Django procura a correspondência no `urls.py` do projeto.
3. Se encontrar um `include`, procura no `urls.py` do app correspondente.
4. Quando encontra a rota, executa a view associada.


#### Resumindo

- **Arquivos e diretórios**: Organizam o código e as configurações do projeto.
- **Rotas (URLs)**: Definem qual view será executada para cada endereço acessado.
- **Views**: Recebem as requisições, processam dados e retornam respostas (geralmente templates renderizados).

Essa estrutura torna o desenvolvimento com Django organizado, escalável e fácil de manter.

---

# **2. Arquitetura MTV do Django**

O Django utiliza o padrão de arquitetura **MTV**, que significa **Model-Template-View**. Esse padrão é semelhante ao conhecido MVC (Model-View-Controller), mas com algumas adaptações para o contexto do framework. Veja como cada componente funciona:


## Model (Modelo)

O **Model** representa a estrutura dos dados da aplicação. Ele define as tabelas do banco de dados, seus campos e as regras de negócio associadas. Cada modelo é uma classe Python que herda de `django.db.models.Model`.

**Exemplo:**
```python
from django.db import models

class Doador(models.Model):
    nome = models.CharField(max_length=100)
    cpf = models.CharField(max_length=11)
    # outros campos...


## Template (Template)

O **Template** é responsável pela camada de apresentação, ou seja, pelo que o usuário vê na interface. Os templates são arquivos HTML (podendo conter tags especiais do Django) que exibem os dados enviados pelas views.

**Exemplo de template (`listar_doadores.html`):**
```html
<h1>Lista de Doadores</h1>
<ul>
  {% for doador in page_obj %}
    <li>{{ doador.nome }} - {{ doador.cpf }}</li>
  {% endfor %}
</ul>
```


## View (Visão)

A **View** é responsável por receber as requisições do usuário, processar os dados (consultando os modelos, por exemplo) e retornar uma resposta (geralmente um template renderizado). No Django, as views são funções ou classes Python.

**Exemplo:**
```python
from django.shortcuts import render
from .models import Doador

def listar_doadores(request):
    doadores = Doador.objects.all().order_by('nome')
    return render(request, 'listar_doadores.html', {'page_obj': doadores})
```




### Como funciona o fluxo MTV?

1. **Usuário faz uma requisição** (ex: acessa uma URL).
2. **View** recebe a requisição, processa os dados (usando os Models) e seleciona o Template adequado.
3. **Template** é renderizado com os dados fornecidos pela View.
4. **Resposta** é enviada ao navegador do usuário.



#### Resumindo

- **Model:** Gerencia os dados e regras de negócio.
- **Template:** Define a apresentação dos dados.
- **View:** Faz a ligação entre Model e Template, processando as requisições.

Essa separação facilita a manutenção, reutilização e organização do código em projetos Django.

---